To run this code, place the original dataset in Comfy_Data Directory. The code runs for 31mins, in order to clean dataset. 

How the Code Does This: <br>
Split the data into two groups: <br>
    •   Purchases → where Sale_Return == 1 <br>
	•   Returns → where Sale_Return == 0 <br>
<br>
For each return: <br>
	•	Look through the list of previous purchases made by the same customer for the same product (SKU). <br>
	•	Find a purchase made before the return date. <br>
	•	Check if the purchase has enough remaining quantity to match the return quantity. <br>
	•	If yes: <br>
	    •	Link the return to that purchase.<br>
	    •	Reduce the purchase’s remaining quantity (so it won’t be reused again for another return beyond its quantity).<br>
	    •	Mark the purchase as “returned” and store the return’s Receipt_id.<br>
<br>
After processing: <br>
	•	The purchase rows will have: <br>
	    •	A return_flag = True <br>
	    •	A list of return receipt_ids linked to that purchase <br>
	    •	The return rows will have: <br>
	        •	A matched_purchase_receipt_id showing which purchase it belongs to  

In [1]:
import pandas as pd

In [ ]:
transactions = pd.read_csv('Comfy_Data/Transactions.csv', sep = ';')

In [5]:
# Convert 'Sale_Return', 'Client_id', 'Credit', and 'SKU_id', 'Revenue', 'Quantity'  to numeric types
transactions['Revenue'] = pd.to_numeric(transactions['Revenue'], errors='coerce')
transactions['Quantity'] = pd.to_numeric(transactions['Quantity'], errors='coerce')
transactions['Sale_Return'] = pd.to_numeric(transactions['Sale_Return'], errors='coerce')
transactions['Client_id'] = pd.to_numeric(transactions['Client_id'], errors='coerce')
transactions['Credit'] = pd.to_numeric(transactions['Credit'], errors='coerce')
transactions['SKU_id'] = pd.to_numeric(transactions['SKU_id'], errors='coerce')

#Convert to date time 
transactions['Date'] = pd.to_datetime(transactions['Date'], errors='coerce')

In [ ]:
# Separate purchases and returns
purchases = transactions[transactions['Sale_Return'] == 1].copy()
returns = transactions[transactions['Sale_Return'] == 0].copy()

purchases['remaining_qty'] = purchases['Quantity']
purchases['return_flag'] = False
purchases['return_receipt_ids'] = [[] for _ in range(len(purchases))]

returns['matched'] = False
returns['matched_purchase_receipt_id'] = None

# Revenue per unit for tighter matching
purchases['rev_per_unit'] = purchases['Revenue'] / purchases['Quantity']
returns['rev_per_unit'] = abs(returns['Revenue'] / returns['Quantity'])

# Match returns to purchases with Partial returns
for idx, ret in returns.iterrows():
    matches = purchases[
        (purchases['Client_id'] == ret['Client_id']) &
        (purchases['SKU_id'] == ret['SKU_id']) &
        (purchases['Date'] < ret['Date']) &
        (purchases['remaining_qty'] >= ret['Quantity']) &
        (abs(purchases['rev_per_unit'] - ret['rev_per_unit']) < 1e-2)  
    ].sort_values(by='Date', ascending=True)  # Oldest first

    if not matches.empty:
        match_idx = matches.index[0]
        purchases.at[match_idx, 'remaining_qty'] -= ret['Quantity']
        purchases.at[match_idx, 'return_flag'] = True
        purchases.at[match_idx, 'return_receipt_ids'].append(ret['Receipt_id'])
        returns.at[idx, 'matched'] = True
        returns.at[idx, 'matched_purchase_receipt_id'] = purchases.at[match_idx, 'Receipt_id']

transactions = transactions.merge(
    purchases[['Receipt_id', 'return_flag', 'return_receipt_ids']],
    on='Receipt_id', how='left'
)

In [13]:
transactions.columns

Index(['City', 'Date', 'Receipt_id', 'Sale_Return', 'Client_id', 'Channel',
       'Platform', 'Credit', 'SKU_id', 'Business', 'Brand', 'Category',
       'Region', 'Revenue', 'Quantity', 'City_On_Off', 'Category_Ukr',
       'return_flag', 'return_receipt_ids'],
      dtype='object')

In [7]:
returned_purchases = purchases[purchases['return_flag'] == True]
print(returned_purchases[['Receipt_id', 'Client_id', 'SKU_id', 'Quantity', 'remaining_qty', 'return_receipt_ids']])

                      Receipt_id  Client_id   SKU_id  Quantity  remaining_qty  \
47       17.09.2024-Ч002001-0039  635805339  2644845       1.0            2.0   
63         07.09.2023-ЧDNZ2-0020  689241200  1114847       1.0            2.0   
149        27.03.2023-ЧDNZ2-0008  675272050  1961322       1.0            2.0   
293      21.12.2024-Ч002026-0034  677334771  2754877       1.0            2.0   
464      31.12.2024-Ч002001-0053  976224604  1033992       1.0            2.0   
...                          ...        ...      ...       ...            ...   
2679488    04.01.2025-ЧMIR2-0273  983663224     7172       1.0            2.0   
2679824  02.03.2025-Ч029307-0661  981709336     7172       1.0            2.0   
2679825  02.03.2025-Ч029307-0661  981709336  3201124       1.0            2.0   
2679892  05.03.2025-Ч029307-1588  960835042  2325154       1.0            2.0   
2679893  05.03.2025-Ч029307-1588  960835042     7139       1.0            2.0   

                   return_r

In [9]:
returned_purchases = purchases[purchases['return_flag'] == True]
print(returned_purchases[['Receipt_id', 'Client_id', 'SKU_id', 'Quantity', 'remaining_qty', 'return_receipt_ids']])

                      Receipt_id  Client_id   SKU_id  Quantity  remaining_qty  \
47       17.09.2024-Ч002001-0039  635805339  2644845       1.0            2.0   
63         07.09.2023-ЧDNZ2-0020  689241200  1114847       1.0            2.0   
149        27.03.2023-ЧDNZ2-0008  675272050  1961322       1.0            2.0   
293      21.12.2024-Ч002026-0034  677334771  2754877       1.0            2.0   
464      31.12.2024-Ч002001-0053  976224604  1033992       1.0            2.0   
...                          ...        ...      ...       ...            ...   
2679488    04.01.2025-ЧMIR2-0273  983663224     7172       1.0            2.0   
2679824  02.03.2025-Ч029307-0661  981709336     7172       1.0            2.0   
2679825  02.03.2025-Ч029307-0661  981709336  3201124       1.0            2.0   
2679892  05.03.2025-Ч029307-1588  960835042  2325154       1.0            2.0   
2679893  05.03.2025-Ч029307-1588  960835042     7139       1.0            2.0   

                   return_r

In [10]:
matched_returns = returns[returns['matched'] == True]
print(matched_returns[['Receipt_id', 'Client_id', 'SKU_id', 'Quantity', 'matched_purchase_receipt_id']])

                         Receipt_id  Client_id   SKU_id  Quantity  \
10          18.09.2024-П002002-0008  635805339  2644845      -1.0   
159      01.04.2023-ВНDNZ-000000670  675272050  1961322      -1.0   
195         24.11.2023-П002007-0002  508855828  1660367      -1.0   
253         17.12.2023-П002007-0005  972916768  2549538      -1.0   
291         20.01.2024-П002007-0001  636372701   979348      -1.0   
...                             ...        ...      ...       ...   
2679307     18.02.2025-П029601-0009  961827242  1356353      -1.0   
2679309     18.02.2025-П029601-0009  961827242     7139      -1.0   
2679310     18.02.2025-П029601-0009  961827242  2688619      -1.0   
2679891     07.03.2025-П029618-0008  960835042  2325154      -1.0   
2679894     07.03.2025-П029618-0008  960835042     7139      -1.0   

        matched_purchase_receipt_id  
10          17.09.2024-Ч002001-0039  
159           27.03.2023-ЧDNZ2-0008  
195         20.11.2023-Ч002002-0022  
253         16.12.2

In [11]:
unmatched_returns = returns[returns['matched'] == False]
print(unmatched_returns[['Receipt_id', 'Client_id', 'SKU_id', 'Quantity', 'Date']])

                         Receipt_id  Client_id   SKU_id  Quantity       Date
78          06.10.2024-П002002-0001  979352501  2981833      -1.0 2024-10-06
224         22.07.2024-П002007-0007  988938232  2486762       NaN 2024-07-22
349         06.06.2024-П002007-0002  676352297  1043666      -1.0 2024-06-06
438         24.01.2025-П002007-0006  671119002  1828252      -1.0 2025-01-24
453         19.05.2024-П002007-0002  968861131  1627263      -1.0 2024-05-19
...                             ...        ...      ...       ...        ...
2628511  16.11.2024-ВНMIR-000001598  974500461  2686493      -1.0 2024-11-16
2647544  31.05.2023-ВНMIR-000000649  671380728  1732823      -1.0 2023-05-31
2659007  04.01.2024-ВНLLR-000000068  507389902  2486415       NaN 2024-01-04
2661731  25.09.2023-ВНMIR-000001794  682875945     7172      -1.0 2023-09-25
2661733  25.09.2023-ВНMIR-000001794  682875945  1647453      -1.0 2023-09-25

[14970 rows x 5 columns]


In [14]:
# Filter purchases that were completely returned (i.e., remaining_qty == 0)
fully_returned_purchases = purchases[purchases['remaining_qty'] == 0]

# Get their Receipt IDs
fully_returned_ids = fully_returned_purchases['Receipt_id'].tolist()

# Now remove those rows from the main df
df_cleaned = transactions[~transactions['Receipt_id'].isin(fully_returned_ids)]

# Optional: Reset index
df_cleaned = df_cleaned.reset_index(drop=True)

In [15]:
df_cleaned.to_csv('Comfy_Data/transactions_without_fully_returned_purchases.csv', index=False)